# Dynamic Staffing Optimization for Advertiser Support

This notebook implements a full solution for dynamically planning agent staffing in 2025. We combine:

- A (dummy) forecast of eligible advertiser sign-ups per month (using historical data as an example).
- A dynamic programming (DP) formulation inspired by supply chain inventory management that decides, month-by-month, how many agents to hire or fire.

The DP balances the revenue uplift from serving advertisers against the cost of idle capacity, unmet demand, and firing agents (with associated firing costs).

Each agent can support up to 10 advertisers, and new hires require a 1-month ramp-up (i.e. they are only effective in the following month).

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np

# For demonstration purposes, we create a dummy forecast for eligible advertiser sign-ups.
simulation_months = pd.date_range(start='2025-01-01', end='2025-12-01', freq='MS')

# Dummy forecast data for USA (number of eligible sign-ups per month)
forecast_usa = [20, 25, 30, 28, 22, 35, 40, 38, 32, 30, 27, 25]

print("Forecast for USA (eligible sign-ups):")
for m, d in zip(simulation_months, forecast_usa):
    print(m.strftime('%Y-%m'), ":", d)

Forecast for USA (eligible sign-ups):
2025-01 : 20
2025-02 : 25
2025-03 : 30
2025-04 : 28
2025-05 : 22
2025-06 : 35
2025-07 : 40
2025-08 : 38
2025-09 : 32
2025-10 : 30
2025-11 : 27
2025-12 : 25


## Dynamic Programming Formulation

We now model the staffing decision as a DP problem. Consider the following:

- **State:** \( I_t \) = number of active agents (each with capacity 10) in month \( t \).
- **Decisions:** In month \( t \), decide how many agents to hire (\( h_t \)) and fire (\( f_t \)). Note that hires become effective only in the following month.
- **Immediate Reward:**
  - **Revenue:** We earn \( r \) per advertiser served (up to a capacity of \( 10 \times I_t \)).
  - **Idle Cost:** If capacity exceeds demand \( D_t \), we incur a cost \( c_{idle} \) per unused capacity unit.
  - **Shortage Cost:** If capacity is insufficient, we incur a penalty \( c_{short} \) per unmet capacity unit.
  - **Firing Cost:** Firing an agent costs a fixed amount.

The state transition for month \( t+1 \) is:

 \( I_{t+1} = I_t - f_t + h_t \)

The DP recursion is then:

 \( V(t, I_t) = \max_{h_t, f_t}\{ R_t(I_t, D_t) - (\text{firing cost}) + V(t+1, I_t - f_t + h_t) \} \)

with the terminal condition \( V(T+1, I) = 0 \).

In [ ]:
# Define parameters for the DP and cost functions
params = {
    'T': 12,                       # Total months (2025)
    'agent_capacity': 10,          # Each agent supports 10 advertisers
    'r': 100,                      # Revenue per advertiser served
    'c_idle': 5,                   # Idle cost per unused capacity unit
    'c_short': 20,                 # Shortage cost per unmet capacity unit
    'max_hire': 5,                 # Maximum agents that can be hired in one month
    'annual_salary': 80000,
    # Firing cost: 40% of annual salary prorated monthly
    'firing_cost': 0.4 * 80000 / 12
}

def compute_immediate_reward(I, D, params):
    """
    Compute immediate reward for a month given current active agents I and demand D.

    - Capacity = I * (agent capacity)
    - Revenue = r * min(capacity, D)
    - Idle cost = c_idle * max(capacity - D, 0)
    - Shortage cost = c_short * max(D - capacity, 0)
    """
    capacity = I * params['agent_capacity']
    served = min(capacity, D)
    revenue = params['r'] * served
    idle_cost = params['c_idle'] * max(capacity - D, 0)
    shortage_cost = params['c_short'] * max(D - capacity, 0)
    return revenue - idle_cost - shortage_cost

def dp(t, I, forecast, params, memo, decision):
    """
    Recursive DP function that computes the maximum cumulative net reward from month t to T,
    given I active agents at the start of month t.

    - forecast: list of demand values (number of eligible advertiser sign-ups) for each month t (1-indexed)
    - I: current active agents (each agent has a capacity of 10)
    - h (hires) and f (fires) are the decisions for month t; note that hires become active in the next month.
    """
    T = params['T']
    if t > T:
        return 0  # Terminal condition: no reward after month T

    if (t, I) in memo:
        return memo[(t, I)]

    best_value = -np.inf
    best_decision = None

    # Loop over possible hiring (h) and firing (f) decisions
    for h in range(params['max_hire'] + 1):
        for f in range(I + 1):  # you can fire up to I agents
            # Immediate reward is computed using current active agents I
            immediate_reward = compute_immediate_reward(I, forecast[t-1], params)

            # Subtract firing cost (firing cost is incurred in the current month)
            immediate_reward -= f * params['firing_cost']

            # Next month's state: agents = current agents - fired + hired
            I_next = I - f + h

            # Recursively compute the future value
            future_value = dp(t + 1, I_next, forecast, params, memo, decision)
            total_value = immediate_reward + future_value

            if total_value > best_value:
                best_value = total_value
                best_decision = (h, f)

    memo[(t, I)] = best_value
    decision[(t, I)] = best_decision
    return best_value

def reconstruct_decisions(t, I, forecast, params, decision):
    """
    Reconstruct the optimal decision path starting from month t with I active agents.
    Returns a list of tuples: (month, active_agents, (hires, fires))
    """
    T = params['T']
    decisions = []
    while t <= T:
        dec = decision.get((t, I), (0, 0))
        decisions.append((t, I, dec))
        h, f = dec
        I = I - f + h
        t += 1
    return decisions

# Set forecast demand for USA using our dummy data
forecast = forecast_usa  # list of 12 values corresponding to months 1 to 12

# Initial number of active agents (e.g., from agent_staffing.csv, assume 5 for USA)
initial_agents = 5

# Run DP
memo = {}
decision = {}
optimal_value = dp(1, initial_agents, forecast, params, memo, decision)

print("Optimal cumulative net reward:", optimal_value)

# Reconstruct and display the optimal decisions path
optimal_decisions = reconstruct_decisions(1, initial_agents, forecast, params, decision)
print("\nOptimal Decisions (month, active_agents, (hires, fires)):")
for d in optimal_decisions:
    print(d)

Optimal cumulative net reward: 33960.0

Optimal Decisions (month, active_agents, (hires, fires)):
(1, 5, (0, 0))
(2, 5, (0, 0))
(3, 5, (0, 0))
(4, 5, (0, 0))
(5, 5, (0, 0))
(6, 5, (0, 0))
(7, 5, (0, 0))
(8, 5, (0, 0))
(9, 5, (0, 0))
(10, 5, (0, 0))
(11, 5, (0, 0))
(12, 5, (0, 0))


## Summary

In this notebook, we:

1. **Forecasted** the number of eligible advertiser sign-ups for USA (dummy data).
2. **Formulated** a dynamic programming model where each month we decide how many agents to hire or fire.
3. **Optimized** the staffing plan to maximize net reward (revenues minus idle, shortage, and firing costs).

You can adjust the forecast data, cost parameters, or constraints as needed for your analysis.

In [ ]:
# prompt: Gurobi Linear

import gurobipy as gp
from gurobipy import GRB

# Define parameters (replace with your actual data)
# appear 30 days
appear_usa = []
initial_agents = 5  # Initial number of active agents

# Fixed values
agent_capacity = 10
finish_period = 60 # days
ramp_up_period = 30 # days

r = 100
c_idle = 5
c_short = 20
max_hire = 5
firing_cost = 0.4 * 80000 / 12
initial_agents = 5

# Create a Gurobi model
model = gp.Model("StaffingOptimization")

# Decision variables
hires = [model.addVar(name=f"hires_{t}", lb=0, ub=max_hire, vtype=GRB.INTEGER) for t in range(T)]
fires = [model.addVar(name=f"fires_{t}", lb=0, vtype=GRB.INTEGER) for t in range(T)]
agents = [model.addVar(name=f"agents_{t}", lb=0, vtype=GRB.INTEGER) for t in range(T)]
served = [model.addVar(name=f"served_{t}", lb=0, vtype=GRB.INTEGER) for t in range(T)]

# Initial condition
model.addConstr(agents[0] == initial_agents)


# Constraints
for t in range(T):
  # Agent balance
  if t > 0:
    model.addConstr(agents[t] == agents[t-1] - fires[t-1] + hires[t-1])
  # Capacity constraint
  model.addConstr(served[t] <= agents[t] * agent_capacity)
  model.addConstr(served[t] <= D[t])


# Objective function
obj = gp.quicksum(r * served[t] - c_idle * max(0, agents[t] * agent_capacity - D[t]) - c_short * max(0, D[t]- agents[t] * agent_capacity) - fires[t] * firing_cost for t in range(T))
model.setObjective(obj, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Print the solution
print("\nOptimal Solution:")
if model.Status == GRB.OPTIMAL:
    for t in range(T):
        print(f"Month {t+1}: Hires = {hires[t].x}, Fires = {fires[t].x}, Agents = {agents[t].x}, Served = {served[t].x}")

    print(f"\nOptimal objective value: {model.ObjVal}")
else:
    print("No optimal solution found")
